In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

In [26]:
# inputs
xs = np.array([-1.0,  0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)

# labels
ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)

## Sequencial Model with MSE loss function

In [17]:
model = keras.Sequential([keras.layers.Dense(1, input_shape = [1])])
model.compile(optimizer=keras.optimizers.SGD(), loss = keras.losses.mse, metrics = ['accuracy'] )

In [18]:
history = model.fit(xs, ys, epochs=300, verbose = False)
print(history)

In [19]:
model.predict([10.0])

1/1 [==============================] - 0s 60ms/step


array([[18.88869]], dtype=float32)

## Functional Model with custom loss function (defined without facilitating hyperparameters)



In [45]:
def custom_huber_loss(y_true, y_pred):
  threshold = 1
  er = y_true - y_pred
  small_error_loss = tf.square(er) / 2
  big_error_loss = threshold * (tf.abs(er) - (0.5 * threshold))
  return tf.where(tf.abs(er) <= threshold, small_error_loss, big_error_loss)

In [49]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
_input = Input(shape=(1))
_output = Dense(1)(_input)
model = Model(inputs = _input, outputs = _output)
model.compile(optimizer = keras.optimizers.SGD(), loss = custom_huber_loss)
history = model.fit(xs, ys, epochs=300, verbose = False)
model.predict([10.0])

1/1 [==============================] - 0s 57ms/step


array([[18.513416]], dtype=float32)

## Functional Model with custom loss function (defined facilitating hyperparameters)

In [63]:
def huber_loss_with_threshold(threshold):
  def huber_loss(y_true, y_pred):
    error = y_true - y_pred
    return tf.where(tf.abs(error) <= threshold, tf.square(error) / 2, threshold * (tf.abs(error) - (0.5 * threshold)))
  return huber_loss

In [64]:
new_model = tf.keras.models.clone_model(model)
new_model.compile(optimizer = keras.optimizers.SGD(), loss = huber_loss_with_threshold(0.9))
new_model.fit(xs, ys, epochs=300, verbose = False)
new_model.predict([10.0])

1/1 [==============================] - 0s 52ms/step


array([[17.995691]], dtype=float32)

## Functional Model with custom loss function (defined facilitating hyperparameters with class)

In [60]:
from tensorflow.keras.losses import Loss
class my_huber_loss(Loss):
  threshold = 1
  def __init__(self, threshold):
    super().__init__()
    self.threshold = threshold
  def call(self, y_true, y_pred):
    error = y_true - y_pred
    return tf.where(tf.abs(error) <= self.threshold, tf.square(error) / 2, self.threshold * (tf.abs(error) - (0.5 * self.threshold)))

In [62]:
new_model = tf.keras.models.clone_model(model)
new_model.compile(optimizer = keras.optimizers.SGD(), loss = my_huber_loss(threshold = 0.9))
new_model.fit(xs, ys, epochs=300, verbose = False)
new_model.predict([10.0])

1/1 [==============================] - 0s 70ms/step


array([[18.131731]], dtype=float32)

[Keras Tuner](https://keras.io/keras_tuner/) can be used to optimize the threshold value.